<a href="https://colab.research.google.com/github/22AlejoL/ALEJO-Y-MI-AMIGO/blob/main/SUDOKUCOMPLETO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import copy

class CSP:
    def __init__(self): # Constructor de la clase CSP
        self.Vars = {} # Representa las variables del problema (en este caso, las celdas del sudoku)
        self.Constraints = {'Dif': [], 'MismoDominio2': [], 'MismoDominio3': [], 'NoRepetido': []} # Contiene las restricciones del problema y la variable
        self.checkReductions = False # Indica si se realizó alguna eliminación en el dominio de las variables.

    # Inicializa el diccionario Vars, asignando a cada celda del tablero de sudoku un conjunto de valores posibles (en este caso, los números del 1 al 9).
    def DominiosVars(self):
        filas = set(range(1, 10))
        columnas = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
        for c in columnas:
            for r in filas:
                self.Vars[f"{c}{r}"] = filas.copy()

    # Esta función asigna un valor específico a una variable (celda del sudoku) en el diccionario Vars.
    def asignarValorEnVars(self, Nombre, Valor):
        self.Vars[Nombre] = {Valor}

    # Esta función lee un archivo que contiene un tablero de sudoku y asigna los valores iniciales a las variables del problema.
    def inicializarTablero(self, nombreArchivo):
        archivo = open(nombreArchivo, "r")
        ListaSudoku = "ABCDEFGHI"
        for r in range(1, 10):
            for c in ListaSudoku:
                cur = int(archivo.readline())
                if cur < 10:
                    self.asignarValorEnVars(f"{c}{r}", cur)

    # Esta función genera las restricciones correspondientes a las subcuadrículas del tablero de sudoku (también conocidas como "ventanas" en el código) y las agrega al diccionario Constraints.
    def restriccionesVentanas(self, claveCons, inicioRango, finRango):
        tripletas = ["ABC", "DEF", "GHI"]
        for t in tripletas:
            l = []
            for c in t:
                for i in range(inicioRango, finRango):
                    l.append(str(c) + str(i))
            self.Constraints[claveCons].append(l)

    # Similar a la función anterior, esta genera las restricciones correspondientes a las columnas y las agrega al diccionario Constraints.
    def restriccionesColumnas(self, claveCons):
        indicesCols = "ABCDEFGHI"
        for i in indicesCols:
            l = []
            for j in range(1, 10):
                l.append(str(i) + str(j))
            self.Constraints[claveCons].append(l)

    # Esta función genera las restricciones correspondientes a las filas del tablero de sudoku y las agrega al diccionario Constraints.
    def restriccionesFilas(self, claveCons):
        indicesCols = "ABCDEFGHI"
        for i in range(1, 10):
            l = []
            for j in indicesCols:
                l.append(str(j) + str(i))
            self.Constraints[claveCons].append(l)

    # Esta función se encarga de agregar todas las restricciones (filas, columnas y subcuadrículas) al diccionario Constraints.
    def estructurasRestriccion(self, claveCons):
        self.Constraints[claveCons] = []
        self.restriccionesFilas(claveCons)
        self.restriccionesColumnas(claveCons)
        inicioRango = [1, 4, 7]
        finRango = [4, 7, 10]
        for indice in range(3):
            self.restriccionesVentanas(claveCons, int(inicioRango[indice]), int(finRango[indice]))

    # Esta función implementa una restricción del sudoku que se encarga de asignar valores a las variables que solo tienen una opción posible en su dominio.
    def SinRepeticiones(self):
        for restriccion in self.Constraints['NoRepetido']:
            todos_valores = set()
            for var in restriccion:
                todos_valores.update(self.Vars[var])
            for valor in todos_valores:
                cuenta = 0
                for var in restriccion:
                    if valor in self.Vars[var]:
                        cuenta += 1
                        var_unico = var
                if (cuenta == 1) and not (self.numElemento(var_unico) == 1):
                    self.Vars[var_unico] = {valor}
                    self.checkReductions = True

    # Esta función implementa una restricción específica del sudoku que se encarga de eliminar valores del dominio de otras variables cuando una variable tiene un único valor posible.
    def diferencia(self, lista, indice):
        for var in lista:
            if self.numElemento(var) == 1:
                elemento = self.Vars[var]
                self.eliminarElementoEnVar(var, lista, elemento.copy())
                self.eliminarVarEnRestriccion('Dif', var, indice)

    # Esta función implementa una restricción del sudoku que se encarga de eliminar valores del dominio de otras variables cuando dos variables tienen exactamente dos valores en su dominio y estos son iguales.
    def MismoDominio2(self):
        for restriccion in self.Constraints['MismoDominio2']:
            for var_1 in restriccion:
                if self.numElemento(var_1) == 2:
                    for var_2 in restriccion:
                        if var_1 == var_2:
                            pass
                        else:
                            if self.Vars[var_1] == self.Vars[var_2]:
                                lista_valores = list(self.Vars[var_1])
                                val_1 = lista_valores[0]
                                val_2 = lista_valores[1]

                                for valParaEliminar in restriccion:
                                    if (valParaEliminar == var_1) or (valParaEliminar == var_2):
                                        pass
                                    else:
                                        lenAntes = len(self.Vars[valParaEliminar])
                                        self.Vars[valParaEliminar].discard(val_1)
                                        self.Vars[valParaEliminar].discard(val_2)
                                        lenDespues = len(self.Vars[valParaEliminar])
                                        if lenAntes > lenDespues:
                                            self.checkReductions = True

    # Similar a la función anterior, pero para cuando tres variables tienen exactamente tres valores en su dominio y estos son iguales.
    def MismoDominio3(self):
        for restriccion in self.Constraints['MismoDominio3']:
            for var_1 in restriccion:
                if self.numElemento(var_1) == 3:
                    for var_2 in restriccion:
                        if var_1 == var_2:
                            pass
                        else:
                            if self.Vars[var_1] == self.Vars[var_2]:
                                for var_3 in restriccion:
                                    if (var_3 == var_1) or (var_3 == var_2):
                                        pass
                                    else:
                                        if self.Vars[var_3] == self.Vars[var_1]:
                                            lista = list(self.Vars[var_1])
                                            val_1 = lista[0]
                                            val_2 = lista[1]
                                            val_3 = lista[2]

                                            for valParaEliminar in restriccion:
                                                if (valParaEliminar == var_1) or (valParaEliminar == var_2) or (valParaEliminar == var_3):
                                                    pass
                                                else:
                                                    lenAntes = len(self.Vars[valParaEliminar])
                                                    self.Vars[valParaEliminar].discard(val_1)
                                                    self.Vars[valParaEliminar].discard(val_2)
                                                    self.Vars[valParaEliminar].discard(val_3)
                                                    lenDespues = len(self.Vars[valParaEliminar])
                                                    if lenAntes > lenDespues:
                                                        self.checkReductions = True

    # Esta función elimina una variable de una restricción específica del diccionario Constraints.
    def eliminarVarEnRestriccion(self, claveCons, var, i):
        lista = self.Constraints[claveCons][i]
        lista.remove(var)

    # Esta función elimina un elemento del dominio de una variable y actualiza la variable checkReductions si se realiza una eliminación.
    def eliminarElementoEnVar(self, var, lista, elemento):
        elemento_entero = int(elemento.pop())
        for c in lista:
            if c == var:
                pass
            else:
                self.Vars[c].discard(elemento_entero)
                self.checkReductions = True

    # Esta función devuelve el número de elementos en el dominio de una variable específica.
    def numElemento(self, clave):
        return len(self.Vars[clave])

    # Esta función recorre todas las restricciones del problema y aplica las restricciones de diferencia, mismos dominios y no repetidos.
    def recorrerRestriccion(self):
        self.checkReductions = False
        for index in range(len(self.Constraints['Dif'])):
            self.diferencia(self.Constraints['Dif'][index], index)
        self.MismoDominio2()
        self.MismoDominio3()
        self.SinRepeticiones()
        return self.checkReductions

    # Esta función verifica si el sudoku está completamente resuelto (todas las celdas tienen un único valor en su dominio).
    def esta_resuelto(self):
        indicesCols = "ABCDEFGHI"
        for c in indicesCols:
            for i in range(1, 10):
                if not (len(self.Vars[str(c) + str(i)]) == 1):
                    return False
        return True

    # Esta función verifica si el sudoku es consistente en el estado actual del tablero, es decir, si no hay celdas vacías sin posibles valores y no hay números repetidos en filas, columnas o subcuadrículas.
    def consistente_localmente(self):
        indicesCols = "ABCDEFGHI"
        for c in indicesCols:
            for i in range(1, 10):
                if (len(self.Vars[str(c) + str(i)]) == 0):
                    return False
        return True

    # Esta función crea una copia profunda del objeto actual de la clase CSP.
    def copiar(self):
        return copy.deepcopy(self)

    # Esta función implementa el algoritmo de backtracking para resolver el sudoku. Realiza una búsqueda exhaustiva.
    def backTrackingExhaustivo(self, sudoku):
        indicesCols = "ABCDEFGHI"
        romper_bucle_exterior = False
        for c in indicesCols:
            for i in range(1, 10):
                if (len(self.Vars[str(c) + str(i)]) == 2):
                    lista = list(self.Vars[str(c) + str(i)])
                    elemento = lista[0]
                    self.Vars[str(c) + str(i)].discard(elemento)
                    romper_bucle_exterior = True
                    break
            if romper_bucle_exterior:
                break
        if romper_bucle_exterior:
            while self.recorrerRestriccion():
                pass
            if self.consistente_localmente() == False:
                sudoku.Vars[str(c) + str(i)] = {elemento}
                return False
            else:
                return True


sudoku = CSP()
sudoku.DominiosVars()
sudoku.inicializarTablero("TableroSudoku.txt")

sudoku.estructurasRestriccion('NoRepetido')
sudoku.estructurasRestriccion('MismoDominio2')
sudoku.estructurasRestriccion('MismoDominio3')
sudoku.estructurasRestriccion('Dif')

# SE DA SOLUCION AL SUDOKU.
while sudoku.esta_resuelto() == False:

    while sudoku.recorrerRestriccion():
        pass
    if sudoku.esta_resuelto() == False:
        prueba = sudoku.copiar()
        if prueba.backTrackingExhaustivo(sudoku) == False:
            pass
        else:
            sudoku = prueba
# SE MUESTRA EL SUDOKU SOLUCIONADO
print("*-----------------------*")
print("| EL SUDOKU RESUELTO ES |")
print("*-----------------------*")
for r in range(1, 10):
    for c in "ABCDEFGHI":
        print(f"'{c}{r}': {sudoku.Vars[f'{c}{r}']}", end=", ")
    print()


*-----------------------*
| EL SUDOKU RESUELTO ES |
*-----------------------*
'A1': {4}, 'B1': {7}, 'C1': {3}, 'D1': {2}, 'E1': {6}, 'F1': {8}, 'G1': {9}, 'H1': {5}, 'I1': {1}, 
'A2': {6}, 'B2': {5}, 'C2': {8}, 'D2': {1}, 'E2': {9}, 'F2': {3}, 'G2': {2}, 'H2': {4}, 'I2': {7}, 
'A3': {2}, 'B3': {1}, 'C3': {9}, 'D3': {7}, 'E3': {5}, 'F3': {4}, 'G3': {8}, 'H3': {6}, 'I3': {3}, 
'A4': {7}, 'B4': {2}, 'C4': {6}, 'D4': {3}, 'E4': {1}, 'F4': {5}, 'G4': {4}, 'H4': {9}, 'I4': {8}, 
'A5': {8}, 'B5': {9}, 'C5': {1}, 'D5': {6}, 'E5': {4}, 'F5': {7}, 'G5': {3}, 'H5': {2}, 'I5': {5}, 
'A6': {5}, 'B6': {3}, 'C6': {4}, 'D6': {8}, 'E6': {2}, 'F6': {9}, 'G6': {1}, 'H6': {7}, 'I6': {6}, 
'A7': {1}, 'B7': {4}, 'C7': {2}, 'D7': {5}, 'E7': {8}, 'F7': {6}, 'G7': {7}, 'H7': {3}, 'I7': {9}, 
'A8': {9}, 'B8': {6}, 'C8': {7}, 'D8': {4}, 'E8': {3}, 'F8': {1}, 'G8': {5}, 'H8': {8}, 'I8': {2}, 
'A9': {3}, 'B9': {8}, 'C9': {5}, 'D9': {9}, 'E9': {7}, 'F9': {2}, 'G9': {6}, 'H9': {1}, 'I9': {4}, 
